## 환경 설정

In [1]:
import os
from glob import glob

from pprint import pprint
import json

import numpy as np
import pandas as pd

from dotenv import load_dotenv
load_dotenv()

True

## Load Data

In [2]:
# CSV 파일 읽기
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
titanic = pd.read_csv(url)

titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
titanic.shape

(891, 12)

## SQLite Database

In [5]:
import sqlite3

# SQLite 데이터베이스 생성
conn = sqlite3.connect('titanic.db')

# Passengers 테이블 생성 및 데이터 삽입
titanic.to_sql('Passengers', conn, if_exists='replace', index=False,
                dtype={
                    'PassengerId': 'INTEGER PRIMARY KEY',
                    'Survived': 'INTEGER',
                    'Pclass': 'INTEGER',
                    'Name': 'TEXT NOT NULL',
                    'Sex': 'TEXT NOT NULL',
                    'Age': 'FLOAT',
                    'SibSp': 'INTEGER',
                    'Parch': 'INTEGER',
                    'Ticket': 'TEXT',
                    'Fare': 'FLOAT',
                    'Cabin': 'TEXT',
                    'Embarked': 'TEXT'
                })

# 데이터베이스 상태 확인
cursor = conn.cursor()
cursor.execute("SELECT COUNT(*) FROM Passengers")
passenger_count = cursor.fetchone()[0]

print("\n=== 데이터베이스 생성 완료 ===")
print(f"승객 수: {passenger_count}")

# 간단한 통계 확인
print("\n=== 기본 통계 ===")
cursor.execute("SELECT COUNT(*) FROM Passengers WHERE Survived = 1")
survivors = cursor.fetchone()[0]
print(f"생존자 수: {survivors}")

cursor.execute("SELECT COUNT(DISTINCT Pclass) FROM Passengers")
pclass_count = cursor.fetchone()[0]
print(f"선실 등급 수: {pclass_count}")

conn.commit()
conn.close()


=== 데이터베이스 생성 완료 ===
승객 수: 891

=== 기본 통계 ===
생존자 수: 342
선실 등급 수: 3


## LangChain 연동

`(1) DB 스키마 확인`

In [6]:
from langchain_community.utilities import SQLDatabase

# SQLite 데이터베이스 연결
db = SQLDatabase.from_uri("sqlite:///titanic.db")

# 사용 가능한 테이블 목록 출력
print("=== 사용 가능한 테이블 목록 ===")
tables = db.get_usable_table_names()
print(tables)  

=== 사용 가능한 테이블 목록 ===
['Passengers']


In [7]:
# 각 테이블의 스키마 정보 출력
print("\n=== 테이블 스키마 정보 ===")
print(db.get_table_info())


=== 테이블 스키마 정보 ===

CREATE TABLE "Passengers" (
	"PassengerId" INTEGER, 
	"Survived" INTEGER, 
	"Pclass" INTEGER, 
	"Name" TEXT NOT NULL, 
	"Sex" TEXT NOT NULL, 
	"Age" FLOAT, 
	"SibSp" INTEGER, 
	"Parch" INTEGER, 
	"Ticket" TEXT, 
	"Fare" FLOAT, 
	"Cabin" TEXT, 
	"Embarked" TEXT, 
	PRIMARY KEY ("PassengerId")
)

/*
3 rows from Passengers table:
PassengerId	Survived	Pclass	Name	Sex	Age	SibSp	Parch	Ticket	Fare	Cabin	Embarked
1	0	3	Braund, Mr. Owen Harris	male	22.0	1	0	A/5 21171	7.25	None	S
2	1	1	Cumings, Mrs. John Bradley (Florence Briggs Thayer)	female	38.0	1	0	PC 17599	71.2833	C85	C
3	1	3	Heikkinen, Miss. Laina	female	26.0	0	0	STON/O2. 3101282	7.925	None	S
*/


`(2) DB 쿼리 실행`

In [8]:
# 생존자 수 확인
query = """
SELECT COUNT(*) FROM Passengers WHERE Survived = 1
"""
pprint(db.run(query))

'[(342,)]'


`(3) SQL Chain`

In [ ]:
# 사용자 질문(text) -> SQL 쿼리 (sql)
from langchain.chains import create_sql_query_chain

from langchain_ollama import ChatOllama
from langchain_google_genai import ChatGoogleGenerativeAI

ollama_llm = ChatOllama(model="qwen2.5")
gemini_llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

ollama_sql = create_sql_query_chain(llm=ollama_llm, db=db)
gemini_sql = create_sql_query_chain(llm=gemini_llm, db=db)

ollama_query = ollama_sql.invoke({"question": "생존자는 모두 몇 명인가요?"})
gemini_query = gemini_sql.invoke({"question": "생존자는 모두 몇 명인가요?"})

print(ollama_query)
print(gemini_query)

/Users/wiggler/Library/Caches/pypoetry/virtualenvs/rag-master-HVkRb5Tu-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Question: 生存者共有多少人？
SQLQuery: SELECT COUNT("Survived") AS SurvivedCount FROM "Passengers" WHERE "Survived" = 1;
Question: 생존자는 모두 몇 명인가요?
SQLQuery: SELECT COUNT("PassengerId") FROM Passengers WHERE "Survived" = 1


In [10]:
import re

def extract_sql(text):
    # SQLQuery: 이후의 텍스트를 추출하는 패턴
    pattern = r'SQLQuery: (.*)'

    # 정규식으로 추출
    match = re.search(pattern, text)
    if match:
        query = match.group(1)  
        return query
    
    return None

print(extract_sql(ollama_query))
print(extract_sql(gemini_query))

SELECT COUNT("Survived") AS SurvivedCount FROM "Passengers" WHERE "Survived" = 1;
SELECT COUNT("PassengerId") FROM Passengers WHERE "Survived" = 1


In [11]:
db.run(extract_sql(ollama_query))

'[(342,)]'

In [ ]:
db.run(extract_sql(gemini_query))

'[(342,)]'

`(4) QA Chain `

In [13]:
# 쿼리를 직접 실행하는 도구 
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

query_excecuter = QuerySQLDataBaseTool(db=db)
query_excecuter.invoke(extract_sql(gemini_query))

'[(342,)]'

In [16]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda


answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

qa_chain = (
    RunnablePassthrough.assign(query=ollama_sql).assign(
        result=itemgetter("query") | RunnableLambda(extract_sql) | query_excecuter
    )
    | answer_prompt
    | gemini_llm
    | StrOutputParser()
)

qa_chain.invoke({"question": "생존자는 모두 몇 명인가요?"})

'생존자는 342명입니다. \n'

## Gradio 챗봇

In [17]:
import gradio as gr

def predict(message, history):
    response = qa_chain.invoke({"question": message})
    return response

demo = gr.ChatInterface(fn=predict, title="SQL Bot")

demo.launch()

/Users/wiggler/Library/Caches/pypoetry/virtualenvs/rag-master-HVkRb5Tu-py3.11/lib/python3.11/site-packages/gradio/components/chatbot.py:229: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [ ]:
demo.close()